![](https://github.com/kalelmartinho/7daysOfCode/blob/master/7daysOfCode.png?raw=true)

---

Por *Kalel Leonardo Martinho*

# #7DaysOfCode - 1: Data Cleaning and Preparation 🧹

Aplicar processos de tratamento e limpeza de dados identificando inconsistências, como campos que possuem valores nulos ou duplicados, converter campos de data que estão sendo carregados como texto, corrigir valores monetários, nomes incorretos, formatar campos de CNPJ, etc.

---


## Uma primeira análise dos dados

A primeira coisa a se fazer, é carregar as bibliotecas e os conjutos de dados que utilizaremos.
Para esse projeto escolhi trabalhar com os dados dos últimos 12 anos (2010-2022).

In [1]:
import pandas as pd
import numpy as np


url = 'https://www.senado.gov.br/transparencia/LAI/verba/despesa_ceaps_'
#url = 'despesa_ceaps_'

# Perceba que utilizamos o método concat para agrupar o conjunto de dados em um único dataset
dados_ceaps = pd.concat(pd.read_csv(url+str(ano)+'.csv', encoding='ISO-8859-1', sep=';', skiprows=1, decimal=',') for ano in range(2010,2023))
dados_ceaps.reset_index(drop=True, inplace=True) # Nessa linha corrigimos o índice com o método reset_index

Após realizar o processo de importação, é importante fazer uma **exploração inicial dos dados.**
Isso permite que você veja que tudo foi lido corretamente e dá uma ideia do que está acontecendo com os dados. Nesse caso, vamos ver se há algum valor ausente, que será representado com NaN ou None.

In [2]:
dados_ceaps.shape # Utilizamos .shape para descobrirmos quantas linhas e colunas possuímos em nosso conjunto de dados

(286388, 11)

In [3]:
dados_ceaps.head() # Utilizamos .head() para verificar as primeiras 5 linhas do nosso conjunto de dados

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2010,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,CENTRAIS ELÉTRICAS DE RONDÔNIA,045216-3,19/01/2010,NaN,59.08,277219.0
1,2010,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,CENTRAIS ELÉTRICAS DE RONDônia,0454206-1,19/01/2010,NaN,40.47,277218.0
2,2010,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,NaN,12/01/2010,NaN,5000.0,229109.0
3,2010,1,ACIR GURGACZ,Divulgação da atividade parlamentar,09.040.457/0001-02,CENTRAL RONDÔNIA.COM,000003,05/01/2010,NaN,1500.0,268264.0
4,2010,1,ACIR GURGACZ,"Locomoção, hospedagem, alimentação, combustíve...",15.896.509/0001-31,POSTO FORTALEZA,018215,07/01/2010,NaN,30.0,268265.0


In [4]:
dados_ceaps.tail() # Utilizamos .tail() para verificar a ultimas 5 linhas dos nossos dados

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
286383,2022,4,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,FXMSVF,02/04/2022,"Companhia Aérea: LATAM, Localizador: FXMSVF. P...",1361.66,2177783.0
286384,2022,4,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,VLNWSC,02/04/2022,"Companhia Aérea: LATAM, Localizador: VLNWSC. P...",2723.23,2177780.0
286385,2022,4,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,QVOLRY,04/04/2022,"Companhia Aérea: GOL, Localizador: QVOLRY. Pas...",1681.23,2177794.0
286386,2022,4,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,JBHH5Y,06/04/2022,"Companhia Aérea: AZUL, Localizador: JBHH5Y. Pa...",1663.61,2177879.0
286387,2022,4,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,FRFFFI,06/04/2022,"Companhia Aérea: LATAM, Localizador: FRFFFI. P...",2324.0,2177912.0


### Quantas informações ausentes temos?

In [5]:
dados_ceaps.isnull().sum()

ANO                       0
MES                       0
SENADOR                   0
TIPO_DESPESA              0
CNPJ_CPF                  0
FORNECEDOR                0
DOCUMENTO             19502
DATA                      0
DETALHAMENTO         112713
VALOR_REEMBOLSADO         0
COD_DOCUMENTO             2
dtype: int64

Pode ser útil ver qual porcentagem dos valores em nosso conjunto de dados esta faltando para nos dar uma ideia melhor da escala desse problema:

In [6]:
# Porcentagem dos dados ausentes
dados_ceaps.isnull().sum()/dados_ceaps.shape[0]*100

ANO                   0.000000
MES                   0.000000
SENADOR               0.000000
TIPO_DESPESA          0.000000
CNPJ_CPF              0.000000
FORNECEDOR            0.000000
DOCUMENTO             6.809643
DATA                  0.000000
DETALHAMENTO         39.356747
VALOR_REEMBOLSADO     0.000000
COD_DOCUMENTO         0.000698
dtype: float64

### Quantas informações duplicadas temos?

Vamos utilizar a coluna COD_DOCUMENTO para verificar por informações duplicadas

In [7]:
dados_ceaps[dados_ceaps['COD_DOCUMENTO'].duplicated(keep=False)]

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
131284,2015,8,EDUARDO AMORIM,"Passagens aéreas, aquáticas e terrestres nacio...",13.353.495/0001-84,PROPAGTUR - Propag Turismo Ltda,"SEM FATURA"";02/08/2015""","Companhia Aérea: TAM, Localizador: YXGDSJ. Pas...","469,53",2014675.0,NaN
131291,2015,8,EDUARDO AMORIM,"Passagens aéreas, aquáticas e terrestres nacio...",13.353.495/0001-84,PROPAGTUR - Propag Turismo Ltda,"¨¨SEM FATURA"";17/08/2015""","Companhia Aérea: AVIANCA, Localizador: ZNEU9F....","460,45",2016557.0,NaN


Nessas linhas a coluna COD_DOCUMENTO ficou sem dados por conta de uma má formatação do arquivo. A coluna DOCUMENTO ficou com 2 informações separadas por ";"


Como são apenas duas linhas é mais eficiente corrigir manualmente

In [8]:
duplicado = 131284

dados_ceaps.loc[duplicado, 'COD_DOCUMENTO'] = dados_ceaps.loc[duplicado, 'VALOR_REEMBOLSADO']
dados_ceaps.loc[duplicado, 'VALOR_REEMBOLSADO'] = dados_ceaps.loc[duplicado, 'DETALHAMENTO']
dados_ceaps.loc[duplicado, 'DETALHAMENTO'] = dados_ceaps.loc[duplicado, 'DATA']
dados_ceaps.loc[duplicado, 'DATA'] = '02/08/2015'
dados_ceaps.loc[duplicado, 'DOCUMENTO'] = 'SEM FATURA'

duplicado = 131291

dados_ceaps.loc[duplicado, 'COD_DOCUMENTO'] = dados_ceaps.loc[duplicado, 'VALOR_REEMBOLSADO']
dados_ceaps.loc[duplicado, 'VALOR_REEMBOLSADO'] = dados_ceaps.loc[duplicado, 'DETALHAMENTO']
dados_ceaps.loc[duplicado, 'DETALHAMENTO'] = dados_ceaps.loc[duplicado, 'DATA']
dados_ceaps.loc[duplicado, 'DATA'] = '17/08/2015'
dados_ceaps.loc[duplicado, 'DOCUMENTO'] = 'SEM FATURA'

dados_ceaps.loc[[131284, 131291]]

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
131284,2015,8,EDUARDO AMORIM,"Passagens aéreas, aquáticas e terrestres nacio...",13.353.495/0001-84,PROPAGTUR - Propag Turismo Ltda,SEM FATURA,02/08/2015,"Companhia Aérea: TAM, Localizador: YXGDSJ. Pas...","469,53",2014675.0
131291,2015,8,EDUARDO AMORIM,"Passagens aéreas, aquáticas e terrestres nacio...",13.353.495/0001-84,PROPAGTUR - Propag Turismo Ltda,SEM FATURA,17/08/2015,"Companhia Aérea: AVIANCA, Localizador: ZNEU9F....","460,45",2016557.0


In [9]:
dados_ceaps[dados_ceaps['COD_DOCUMENTO'].duplicated(keep=False)]

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO


### Descobrir por que estão faltando dados

Em nossa exploração inicial dados percebemos que apenas as colunas *DOCUMENTO* e *DETALHAMENTO* contém dados vazios e não há duplicações. Vamos excluir também as colunas *DOCUMENTO* e *COD_DOCUMENTO*, pois não acrescentam nas informações sobre os gastos.

A coluna *DETALHAMENTO* nos da detalhes sobre o tipo de despesa do senador. Ela pode ser útil em uma análise exploratória ou aplicação de modelo.
Então podemos preencher os dados vazios da coluna *DETALHAMENTO* com informações contida na coluna *TIPO_DESPESA*

In [10]:
dados_copia = dados_ceaps.copy # Cópia dos dados

dados_ceaps = dados_ceaps.drop(columns = ['DOCUMENTO', 'COD_DOCUMENTO']) # removendo DOCUMENTO E COD_DOCUMENTO

dados_ceaps.loc[dados_ceaps['DETALHAMENTO'].isnull(), 'DETALHAMENTO'] = dados_ceaps[dados_ceaps['DETALHAMENTO'].isnull()]['TIPO_DESPESA'] # preenchendo dados ausentes

In [11]:
# Verificando se ainda existem dados ausentes
dados_ceaps.isnull().sum()

ANO                  0
MES                  0
SENADOR              0
TIPO_DESPESA         0
CNPJ_CPF             0
FORNECEDOR           0
DATA                 0
DETALHAMENTO         0
VALOR_REEMBOLSADO    0
dtype: int64

### Verificando o tipo de dados das nossas colunas utilizando .dtypes()

In [12]:
dados_ceaps.dtypes

ANO                   int64
MES                   int64
SENADOR              object
TIPO_DESPESA         object
CNPJ_CPF             object
FORNECEDOR           object
DATA                 object
DETALHAMENTO         object
VALOR_REEMBOLSADO    object
dtype: object

2 colunas estão com formatos incorretos.
Vamos corrigi-las.

>*DATA* `object` para `DateType`

Em uma primeira análise percebi que muitos dados contidos em DATA estão com formato incorreto, apresentando inconsistências com as colunas 'ANO' e 'MES'

In [13]:
fix_data = pd.to_datetime(dados_ceaps['DATA'], dayfirst=True, errors='coerce')
fix_data[fix_data.isna()]

135720   NaT
137188   NaT
137541   NaT
143923   NaT
143927   NaT
148328   NaT
153609   NaT
163856   NaT
165036   NaT
165151   NaT
169396   NaT
184799   NaT
193933   NaT
194164   NaT
196713   NaT
204631   NaT
219150   NaT
221768   NaT
240687   NaT
261697   NaT
267571   NaT
Name: DATA, dtype: datetime64[ns]

Vamos substituir a coluna referente a data com as informações de 'ANO' e 'MES'

In [15]:
dados_ceaps['DATA'] = dados_ceaps.apply(lambda x: '010'+str(x['MES'])+str(x['ANO']) if x['MES'] < 10 else '01'+str(x['MES'])+str(x['ANO']), axis=1)

dados_ceaps['DATA'] = pd.to_datetime(dados_ceaps['DATA'], format='%d%m%Y').dt.normalize()

Como já criamos uma nova coluna 'DATA' não precisaremos das colunas 'ANO' e 'MES'

In [16]:
dados_ceaps = dados_ceaps.drop(columns = ['ANO', 'MES'])

In [17]:
dados_ceaps.sort_values(by='DATA')

,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DATA,DETALHAMENTO,VALOR_REEMBOLSADO
0,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,CENTRAIS ELÉTRICAS DE RONDÔNIA,2010-01-01,"Aluguel de imóveis para escritório político, c...",59.08
12945,MOZARILDO CAVALCANTI,"Aluguel de imóveis para escritório político, c...",33.000.118/0008-45,Telemar Norte Leste S/A.,2010-01-01,"Aluguel de imóveis para escritório político, c...",126.94
12944,MOZARILDO CAVALCANTI,"Aluguel de imóveis para escritório político, c...",33.000.118/0008-45,Telemar Norte Leste S/A.,2010-01-01,"Aluguel de imóveis para escritório político, c...",80.59
12943,MOZARILDO CAVALCANTI,"Aluguel de imóveis para escritório político, c...",72.820.822/0027-69,Sky Brasil Serviços Ltda.,2010-01-01,"Aluguel de imóveis para escritório político, c...",95.9
12942,MOZARILDO CAVALCANTI,"Aluguel de imóveis para escritório político, c...",33.530.486/0001-29,Empresa Brasileira de Telecomunicações S/A,2010-01-01,"Aluguel de imóveis para escritório político, c...",6.36
...,...,...,...,...,...,...,...
286029,TELMÁRIO MOTA,"Aluguel de imóveis para escritório político, c...",717.750.992-53,HELEN ALICE COSTA DA SILVA,2022-04-01,"Aluguel de imóveis para escritório político, c...",3100.0
284464,JORGINHO MELLO,"Aluguel de imóveis para escritório político, c...",09.603.809/0001-82,Orcali Serviços de Monitoram Eletronivo Ltda,2022-04-01,"Aluguel de imóveis para escritório político, c...",505.78
283726,FLÁVIO ARNS,"Locomoção, hospedagem, alimentação, combustíve...",16.978.175/0001-08,ADRIA VIAGENS E TURISMO LTDA,2022-04-01,"Locomoção, hospedagem, alimentação, combustíve...",214.24
282621,CARLOS PORTINHO,"Passagens aéreas, aquáticas e terrestres nacio...",09.296.295/0001-60,AZUL,2022-04-01,"Companhia Aérea: AZUL, Localizador: IHLG4G. Pa...",2025.19


>*VALOR_REEMBOLSADO* `object` para `float`

In [18]:
def to_dot(value):
    if type(value) == str:
        value = value.replace(',', '.')
        value = value.replace(' ', '')
        return value
    else:
        return value
    
dados_ceaps['VALOR_REEMBOLSADO'] = dados_ceaps['VALOR_REEMBOLSADO'].map(to_dot)  
dados_ceaps.loc[97033, 'VALOR_REEMBOLSADO'] = '1675.55' # tratando dado individual que estava como '1\r\n675.55'

dados_ceaps['VALOR_REEMBOLSADO'] = pd.to_numeric(dados_ceaps['VALOR_REEMBOLSADO'])

In [23]:
dados_ceaps.dtypes

SENADOR                      object
TIPO_DESPESA                 object
CNPJ_CPF                     object
FORNECEDOR                   object
DATA                 datetime64[ns]
DETALHAMENTO                 object
VALOR_REEMBOLSADO           float64
dtype: object

In [20]:
dados_ceaps.head()

,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DATA,DETALHAMENTO,VALOR_REEMBOLSADO
0,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,CENTRAIS ELÉTRICAS DE RONDÔNIA,2010-01-01,"Aluguel de imóveis para escritório político, c...",59.08
1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,CENTRAIS ELÉTRICAS DE RONDônia,2010-01-01,"Aluguel de imóveis para escritório político, c...",40.47
2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,2010-01-01,"Aluguel de imóveis para escritório político, c...",5000.00
3,ACIR GURGACZ,Divulgação da atividade parlamentar,09.040.457/0001-02,CENTRAL RONDÔNIA.COM,2010-01-01,Divulgação da atividade parlamentar,1500.00
4,ACIR GURGACZ,"Locomoção, hospedagem, alimentação, combustíve...",15.896.509/0001-31,POSTO FORTALEZA,2010-01-01,"Locomoção, hospedagem, alimentação, combustíve...",30.00


### Vamos procurar agora por possíveis incosistências em VALOR_REEMBOLSADO

In [ ]:
dados_ceaps.describe()

,VALOR_REEMBOLSADO
count,286388.000000
mean,978.225196
std,2529.897999
min,-1500.000000
25%,100.000000
50%,287.265000
75%,982.767500
max,270000.000000


Perceba que existem valores negativos, vamos tratar esses dados

In [28]:
dados_ceaps.query('VALOR_REEMBOLSADO < -1')

,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DATA,DETALHAMENTO,VALOR_REEMBOLSADO
133386,GLADSON CAMELI,"Aluguel de imóveis para escritório político, c...",63.600.449/0001-00,Arras Adm. de Bens Imóveis Limpeza e Conservaç...,2015-02-01,Contrato de locação de Imóvel comercial do esc...,-1500.0
144296,RANDOLFE RODRIGUES,"Passagens aéreas, aquáticas e terrestres nacio...",02.012.862/0001-60,Tam Linhas Aereas S/A,2015-01-01,"9572100929351 - R$ 1.298,93 (1º bilhete emitid...",-9.0
182570,DÁRIO BERGER,"Contratação de consultorias, assessorias, pesq...",73.228.876/0001-63,TV CLIPAGEM LTDA. EPP.,2017-05-01,Despesa com Monitoramento de Informação Jornal...,-243.4


Como são apenas 3 linhas vamos olhar todos um por um

In [33]:
dados_ceaps.loc[133386, 'DETALHAMENTO']

'Contrato de locação de Imóvel comercial do escritório de apoio do Senador Gladson Cameli no Estado'

In [34]:
dados_ceaps.loc[133386, 'VALOR_REEMBOLSADO'] = dados_ceaps.loc[133386, 'VALOR_REEMBOLSADO'] * -1 #Vamos substituir -1500 por 1500

In [36]:
dados_ceaps.loc[144296, 'DETALHAMENTO']

'9572100929351 - R$ 1.298,93 (1º bilhete emitido) 9572102224403 - R$ 1.418,93 (1ª remarcação) 9572104023443 - R$ 1.498,93 (2ª remarcação) 9572104373256 - R$ 2.412,03 (3ª remarcação - bilhete utilizado)'

Utilizarei o valor referente ao bilhete utilizado como consta no detalhamento R$ 2.412,03

Mais detalhes do gasto em específico pode ser conferido [aqui](https://www6g.senado.leg.br/transparencia/sen/5012/ceaps/8/detalhe/358?mesAno=01/2015)

In [41]:
dados_ceaps.loc[144296, 'VALOR_REEMBOLSADO'] = 2412.03

In [42]:
dados_ceaps.loc[182570, 'DETALHAMENTO']

'Despesa com Monitoramento de Informação Jornalística em Mídia Eletrônica e Imprensa'

In [43]:
dados_ceaps.loc[182570, 'VALOR_REEMBOLSADO'] = dados_ceaps.loc[182570, 'VALOR_REEMBOLSADO'] * -1

In [44]:
dados_ceaps.loc[182570, 'VALOR_REEMBOLSADO']

243.4

### Após tratar os dados podemos exportar esse dataset para um arquivo csv:

In [45]:
dados_ceaps.to_csv('CEAPS_dados_2010_2022.csv', sep=';')